Introduction:

For our project proposal, we will be using is a modified version of the UCI Heart Disease dataset. The UCI Heart Disease dataset includes four databases of the countries: Cleveland, Hungary, Switzerland, and the VA Long Beach. We will be using Cleveland for our prediction because the dataframe contains sufficient observations. The Heart Disease Cleveland dataframe includes the variables: "trestbps" and "thalach", which we will be using to predict our variable of intrest: "num". In the dataframe, "trestbps" refers to the resting blood pressure on admission to the hospital measured in mm Hg, while "thalach" refers to maximum heart rate achieved measured in bps, and "num" means the diagnosis of heart disease with a value of: 0 means no heart disease, 1-4 means the levels of heart disease[1] (We can classify levels of heart disease as 1: Mild, 2: Moderate, 3: Severe 4: Life threatening)[2]. With these resources, we will be answering our question: Can we classify the extent of heart disease using resting blood pressure and maximum heart rate achieved.

